In [2]:
import pandas as pd
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from flair.data import Sentence
from flair.models import TextClassifier

### Data Labelling

In [4]:
# Textblob
textblob_label = pd.read_csv('../1_Preprocessed_Data/phrasebank_preprocessed.csv')

for index, row in textblob_label.iterrows():
    text = str(row['headline_text'])
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity

    if sentiment > 0:
        label = 'positive'
    elif sentiment < 0:
        label = 'negative'
    else:
        label = 'neutral'

    textblob_label.at[index, 'textblob_label'] = label

textblob_label.to_csv('../2_Labelled_Data/textblob_label.csv', index=False) 
textblob_label.head()


headline_text human_label  \
0  [('accord', 'gran', 'compani'), ('gran', 'comp...     neutral   
1  [('technopoli', 'plan', 'develop'), ('plan', '...     neutral   
2  [('intern', 'electron', 'industri'), ('electro...    negative   
3  [('new', 'product', 'plant'), ('product', 'pla...    positive   
4  [('accord', 'compani', 'updat'), ('compani', '...    positive   

  textblob_label  
0        neutral  
1        neutral  
2       negative  
3       negative  
4        neutral

In [5]:
#Vader
vader_label = pd.read_csv('../1_Preprocessed_Data/phrasebank_preprocessed.csv')

analyzer = SentimentIntensityAnalyzer()

for index, row in vader_label.iterrows():
    text = str(row['headline_text']) 

    sentiment_scores = analyzer.polarity_scores(text)

    if sentiment_scores['compound'] > 0:
        label = 'positive'
    elif sentiment_scores['compound'] < 0:
        label = 'negative'
    else:
        label = 'neutral'

    vader_label.at[index, 'vader_label'] = label  

vader_label.to_csv('../2_Labelled_Data/vader_label.csv', index=False) 
vader_label.head()

headline_text human_label vader_label
0  [('accord', 'gran', 'compani'), ('gran', 'comp...     neutral     neutral
1  [('technopoli', 'plan', 'develop'), ('plan', '...     neutral     neutral
2  [('intern', 'electron', 'industri'), ('electro...    negative     neutral
3  [('new', 'product', 'plant'), ('product', 'pla...    positive    positive
4  [('accord', 'compani', 'updat'), ('compani', '...    positive    positive

In [1]:
# BERT
bert_label = pd.read_csv('../1_Preprocessed_Data/phrasebank_preprocessed.csv')

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
num_labels = 3
learning_rate = 2e-5
num_epochs = 5
batch_size = 32

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.eval()

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

predictions = []

for index, row in bert_label.iterrows():
    text = str(row['headline_text'])

    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    encoded_input = {key: tensor.to(device) for key, tensor in encoded_input.items()}

    with torch.no_grad():
        output = model(**encoded_input)

    predicted_label = torch.argmax(output.logits).item()
    predictions.append(predicted_label)

labels = ['negative', 'neutral', 'positive']
predicted_sentiments = [labels[prediction] for prediction in predictions]

bert_label['bert_label'] = predicted_sentiments

bert_label.to_csv('../2_Labelled_Data/bert_label.csv', index=False)
bert_label.head()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

headline_text human_label bert_label
0  [('accord', 'gran', 'compani'), ('gran', 'comp...     neutral   positive
1  [('technopoli', 'plan', 'develop'), ('plan', '...     neutral   positive
2  [('intern', 'electron', 'industri'), ('electro...    negative   positive
3  [('new', 'product', 'plant'), ('product', 'pla...    positive   positive
4  [('accord', 'compani', 'updat'), ('compani', '...    positive   positive

In [7]:
#Flair
flair_label = pd.read_csv('../1_Preprocessed_Data/phrasebank_preprocessed.csv')

classifier = TextClassifier.load('en-sentiment')

flair_label['flair_label'] = ""

for i, headline in enumerate(flair_label['headline_text']):
    sentence = Sentence(str(headline))
    classifier.predict(sentence)
    label = sentence.labels[0].value.lower()
    flair_label.at[i, 'flair_label'] = label

flair_label.to_csv('../2_Labelled_Data/flair_label.csv', index=False) 
flair_label.head()

headline_text human_label flair_label
0  [('accord', 'gran', 'compani'), ('gran', 'comp...     neutral    negative
1  [('technopoli', 'plan', 'develop'), ('plan', '...     neutral    negative
2  [('intern', 'electron', 'industri'), ('electro...    negative    negative
3  [('new', 'product', 'plant'), ('product', 'pla...    positive    negative
4  [('accord', 'compani', 'updat'), ('compani', '...    positive    negative